# Minimal optimization example
Consider $N$ objects $i=0, ..., N-1$ with costs $C_i$. In this file, the object with minimal costs is determined on currently available quantum hardware (simulators). Adaptation to more complex cost functions is straightforward.

The problem can be modeled as a minimization problem of binary variables $X_i\in \{0,1\}$. $X_i$ is set to 1 if object $i$ is chosen and set to 0 if object $i$ is not chosen. The cost function depends on two terms: A term linear in the $X_i$ summing up the costs and a penalty term quadratic in the $X_i$ making sure that only one route is selected. The penalty value $P$ has to be higher than the sum of all costs.
$$ \sum_{i=0}^{N-1} C_i X_i + P \left( \sum_{i=0}^{N-1} X_i - 1 \right)^2$$

Note that one could easily find $i$ such that $C_i$ is minimal on a classical computer. This file merely illustrates the use of D-Wave's quantum annealing simulator and IBM's optimization module.

## Problem definition

First, we implement the cost function in sympy. This allows to easily read out coefficients and to transfer them into the data structures of D-Wave and IBM.

In [1]:
## Cost function specification in sympy
## Specify your variables and cost function here

import sympy as sym

# number of objects
N = sym.symbols('N')

# binary variables X_0, ..., X_{N-1} with values in {0,1}
# use sympy's IndexedBase for indexed variables
X = sym.IndexedBase('X')

# costs
C = sym.IndexedBase('C')

# penalty
P = sym.symbols('P')

# indices
i = sym.symbols('i')

# define cost function
cost_function = (
    sym.Sum( C[i] * X[i] , (i,0,N-1) ) +           # costs for element i
    P * (sym.Sum( X[i], (i, 0, N-1) ) - 1 )**2)    # choose exactly one element

# print cost function
cost_function

P*(Sum(X[i], (i, 0, N - 1)) - 1)**2 + Sum(C[i]*X[i], (i, 0, N - 1))

## Data input
The data can be provided as python lists.

In [2]:
# data input

TestN = 5
TestC = [4,3,7,2,9]
TestP = 50

# set the number of Qubits to TestN

num_qubits = TestN

In [3]:
## transfer the data into a sympy polynomial in the variables X_i
## adjust the dictionaries to your needs

# translation of data into dictionaries for sympy
single_valued_dict = {
    N: TestN,
    P: TestP
}

cost_dict = {
    C[k]: TestC[k] for k in range(len(TestC))
}

# definition of the cost polynomial
cost_poly = sym.Poly(cost_function
                     .subs(single_valued_dict)
                     .doit()
                     .subs(cost_dict),
                     [X[i] for i in range(num_qubits)])
cost_poly

Poly(50*X[0]**2 + 100*X[0]*X[1] + 100*X[0]*X[2] + 100*X[0]*X[3] + 100*X[0]*X[4] - 96*X[0] + 50*X[1]**2 + 100*X[1]*X[2] + 100*X[1]*X[3] + 100*X[1]*X[4] - 97*X[1] + 50*X[2]**2 + 100*X[2]*X[3] + 100*X[2]*X[4] - 93*X[2] + 50*X[3]**2 + 100*X[3]*X[4] - 98*X[3] + 50*X[4]**2 - 91*X[4] + 50, X[0], X[1], X[2], X[3], X[4], domain='ZZ')

## Solving on D-Wave's quantum annealing simulator

First, the associated QUBO-matrix is visualized for the given number $N=TestN$ of Qubits, but for arbitrary costs. The following execution on the quantum hardware simulater is independent of this step.

In [4]:
## QUBO-matrix visualization
## not necessary for the following step

# define sympy polynomial in the variables X_i without costs and penalty
cost_poly_vis = sym.Poly(cost_function
                     .subs({N:TestN})
                     .doit(),
                     [X[i] for i in range(num_qubits)])

# generate QUBO-matrix for the given number of Qubits
Qubo = { (i,j) : cost_poly_vis.coeff_monomial(X[i]**1 * X[j]**1) + ( cost_poly_vis.coeff_monomial(X[i]**1) if (i==j) else 0 )
        for i in range(num_qubits) for j in range(i,num_qubits) }

# visualization as pandas dataframe
import pandas as pd

visualization = [[0 for i in range(num_qubits)] for j in range(num_qubits)]
for key, value in Qubo.items():
    visualization[key[0]][key[1]] = value
pd.DataFrame(visualization)

,0,1,2,3,4
0,-P + C[0],2*P,2*P,2*P,2*P
1,0,-P + C[1],2*P,2*P,2*P
2,0,0,-P + C[2],2*P,2*P
3,0,0,0,-P + C[3],2*P
4,0,0,0,0,-P + C[4]


Second, the QUBO-matrix with purely non-symbolic entries is created and the problem is executed on D-Wave's quantum annealing simulator.

In [5]:
## execution on D-Wave's quantum annealing simulator

# generate QUBO-matrix for the given number of Qubits
Qubo = { (i,j) : (cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
                  + ( cost_poly.coeff_monomial(X[i]**1) if (i==j) else 0 ))
        for i in range(num_qubits)
        for j in range(i,num_qubits) }

# execution on D-Wave simulator
from dwave_qbsolv import QBSolv
result_dwave = QBSolv().sample_qubo(Qubo)

# print result
print("D-Wave simulator: " + str(list(result_dwave.samples())))

D-Wave simulator: [{0: 0, 1: 0, 2: 0, 3: 1, 4: 0}]


## Solving with IBM's optimization module
First, the data are transfered into IBM's optimization data structure and displayed.

In [6]:
import qiskit
from qiskit.aqua.algorithms import QAOA
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer

# generate qiskit's cost function
qiskit_cost_function = qiskit.optimization.QuadraticProgram()

# define qiskit variables
for i in range(num_qubits):
    qiskit_cost_function.binary_var('X_' + str(i))

# specify qiskit cost function
qiskit_cost_function.minimize(
    linear = [int(cost_poly.coeff_monomial(X[i]**1)) for i in range(num_qubits)],
    quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(num_qubits)
        for j in range(i,num_qubits)
    })

# print qiskit cost function
print(qiskit_cost_function.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 96 X_0 - 97 X_1 - 93 X_2 - 98 X_3 - 91 X_4 + [ 100 X_0^2 + 200 X_0*X_1
      + 200 X_0*X_2 + 200 X_0*X_3 + 200 X_0*X_4 + 100 X_1^2 + 200 X_1*X_2
      + 200 X_1*X_3 + 200 X_1*X_4 + 100 X_2^2 + 200 X_2*X_3 + 200 X_2*X_4
      + 100 X_3^2 + 200 X_3*X_4 + 100 X_4^2 ]/2
Subject To

Bounds
 0 <= X_0 <= 1
 0 <= X_1 <= 1
 0 <= X_2 <= 1
 0 <= X_3 <= 1
 0 <= X_4 <= 1

Binaries
 X_0 X_1 X_2 X_3 X_4
End



Second, select the backend. Here, two simulator backends and a real quantum computer are chosen.

In [7]:
# local simulator
backend_local_simulator = qiskit.BasicAer.get_backend('statevector_simulator')

# import IBM account
# for this, you first need to save your IBMQ token, see
# https://qiskit.org/documentation/install.html
from qiskit import IBMQ
provider = IBMQ.load_account()

# 5 qubit quantum computer
backend_quantum = provider.get_backend('ibmq_london')

# 32 qubit online simulator 
backend_online_simulator = provider.get_backend('ibmq_qasm_simulator')

# print selectable backends
# see https://quantum-computing.ibm.com/
# for availability, queue length and maintanance 
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

Finally, solve the problem on all backends using QAOA.

In [8]:
# some round() function in qiskit is deprecated
# turn off the corresponding warning
import warnings
warnings.filterwarnings('ignore')

In [9]:
# execute QAOA on local simulator
qaoa1 = QAOA(quantum_instance = backend_local_simulator)
optimizer1 = MinimumEigenOptimizer(qaoa1)
result1 = optimizer1.solve(qiskit_cost_function)
print("QAOA on local simulator: ", result1)

QAOA on local simulator:  x=[0.0,0.0,0.0,1.0,0.0], fval=-48.0


In [10]:
# execute QAOA on quantum hardware
qaoa2 = QAOA(quantum_instance = backend_quantum)
optimizer2 = MinimumEigenOptimizer(qaoa2)
result2 = optimizer2.solve(qiskit_cost_function)
print("QAOA on quantum computer: ", result2)

The skip Qobj validation does not work for IBMQ provider. Disable it.


QAOA on quantum computer:  x=[0.0,0.0,0.0,1.0,0.0], fval=-48.0


In [11]:
# execute QAOA on online simulator
qaoa3 = QAOA(quantum_instance = backend_online_simulator)
optimizer3 = MinimumEigenOptimizer(qaoa3)
result3 = optimizer3.solve(qiskit_cost_function)
print("QAOA on online simulator: ", result3)

The skip Qobj validation does not work for IBMQ provider. Disable it.


QAOA on online simulator:  x=[0.0,0.0,0.0,1.0,0.0], fval=-48.0
